In [26]:
import caffe
import numpy as np

In [27]:
def EditFcnProto(templateFile, height, width):
    with open(templateFile, 'r') as ft:
        template = ft.read()
        outFile = 'DehazeNet/DehazeNetFcn.prototxt'
        with open(outFile, 'w') as fd:
            fd.write(template.format(height_15=height+15, width_15=width+15, height_11=height+11, width_11=width+11))

In [28]:
templateFile = 'DehazeNet/DehazeFcnTemplate.prototxt'
EditFcnProto(templateFile, 525,600)

In [30]:
caffe.set_mode_cpu()
net = caffe.Net('DehazeNet/DehazeNet.prototxt', 'DehazeNet/DehazeNet.caffemodel', caffe.TEST)
net_full_conv = caffe.Net('DehazeNet/DehazeNetFcn.prototxt', 'DehazeNet/DehazeNet.caffemodel', caffe.TEST)
net_full_conv.params['ip1-conv'][0].data.flat = net.params['ip1'][0].data.flat
net_full_conv.params['ip1-conv'][1].data[...] = net.params['ip1'][1].data
im = caffe.io.load_image("DehazeNet/img/canon.jpg") * 255
print(im.shape)
npad = ((7,8), (7,8), (0,0))
print(im.shape)
im = np.pad(im, npad, 'symmetric')
print(im.shape)
transformers = caffe.io.Transformer({'data': net_full_conv.blobs['data'].data.shape})
print(net_full_conv.blobs['data'].data.shape)
transformers.set_transpose('data', (2,0,1))
transformers.set_channel_swap('data', (2,1,0))
print(im.shape)
print(np.array([transformers.preprocess('data', im-0.2)]).shape)

(525, 600, 3)
(525, 600, 3)
(540, 615, 3)
(1, 3, 540, 615)
(540, 615, 3)
(1, 3, 540, 615)


In [31]:
import caffe
import numpy as np
import argparse
import os

def extract_caffe_model(model, weights, output_path):
    """extract caffe model's parameters to numpy array, and write them to files
        Args:
            model: path of '.prototxt'
            weights: path of '.caffemodel'
            output_path: output path of numpy params 
        Returns:
        None
    """
    net = caffe.Net(model, caffe.TEST)
    net.copy_from(weights)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    for item in net.params.items():
        name, layer = item
        
        num = 0
        for p in net.params[name]:
            np.save(output_path + '/' + str(name).replace('/', '-') + '_' + str(num), p.data)
            num += 1


In [32]:
path_to_model = "DehazeNet/DehazeNetFcn.prototxt"
path_to_weights = "DehazeNet/DehazeNet.caffemodel"
extract_caffe_model(path_to_model, path_to_weights, "DehazeNet/numpy_parameters_for_updated_template")


In [37]:
np.save("DehazeNet/numpy_parameters_for_updated_template" + '/' + 'ip1-conv-transformed' + '_' + str(0), net_full_conv.params['ip1-conv'][0].data)
np.save("DehazeNet/numpy_parameters_for_updated_template" + '/' + 'ip1-conv-transformed' + '_' + str(1), net_full_conv.params['ip1-conv'][1].data)

In [38]:
print(np.load("DehazeNet/numpy_parameters_for_updated_template/ip1-conv-transformed_0.npy").shape)
print(np.load("DehazeNet/numpy_parameters_for_updated_template/ip1-conv-transformed_1.npy").shape)

(1, 64, 5, 5)
(1,)


In [1]:
VGG_MEAN = [103.939, 116.779, 123.68]


class Vgg16:
    def __init__(self, vgg16_npy_path=None):
        if vgg16_npy_path is None:
            path = inspect.getfile(Vgg16)
            path = os.path.abspath(os.path.join(path, os.pardir))
            path = os.path.join(path, "vgg16.npy")
            vgg16_npy_path = path
            print(path)

        self.data_dict = np.load(vgg16_npy_path, encoding='latin1').item()
        print("npy file loaded")

    def build(self, rgb):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """

        start_time = time.time()
        print("build model started")
        rgb_scaled = rgb * 255.0

        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        assert red.get_shape().as_list()[1:] == [224, 224, 1]
        assert green.get_shape().as_list()[1:] == [224, 224, 1]
        assert blue.get_shape().as_list()[1:] == [224, 224, 1]
        bgr = tf.concat(axis=3, values=[
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])
        assert bgr.get_shape().as_list()[1:] == [224, 224, 3]

        self.conv1_1 = self.conv_layer(bgr, "conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1, "conv1_2")
        self.pool1 = self.max_pool(self.conv1_2, 'pool1')

        self.conv2_1 = self.conv_layer(self.pool1, "conv2_1")
        self.conv2_2 = self.conv_layer(self.conv2_1, "conv2_2")
        self.pool2 = self.max_pool(self.conv2_2, 'pool2')

        self.conv3_1 = self.conv_layer(self.pool2, "conv3_1")
        self.conv3_2 = self.conv_layer(self.conv3_1, "conv3_2")
        self.conv3_3 = self.conv_layer(self.conv3_2, "conv3_3")
        self.pool3 = self.max_pool(self.conv3_3, 'pool3')

        self.conv4_1 = self.conv_layer(self.pool3, "conv4_1")
        self.conv4_2 = self.conv_layer(self.conv4_1, "conv4_2")
        self.conv4_3 = self.conv_layer(self.conv4_2, "conv4_3")
        self.pool4 = self.max_pool(self.conv4_3, 'pool4')

        self.conv5_1 = self.conv_layer(self.pool4, "conv5_1")
        self.conv5_2 = self.conv_layer(self.conv5_1, "conv5_2")
        self.conv5_3 = self.conv_layer(self.conv5_2, "conv5_3")
        self.pool5 = self.max_pool(self.conv5_3, 'pool5')

        self.fc6 = self.fc_layer(self.pool5, "fc6")
        assert self.fc6.get_shape().as_list()[1:] == [4096]
        self.relu6 = tf.nn.relu(self.fc6)

        self.fc7 = self.fc_layer(self.relu6, "fc7")
        self.relu7 = tf.nn.relu(self.fc7)

        self.fc8 = self.fc_layer(self.relu7, "fc8")

        self.prob = tf.nn.softmax(self.fc8, name="prob")

        self.data_dict = None
        print(("build model finished: %ds" % (time.time() - start_time)))

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            filt = self.get_conv_filter(name)

            conv = tf.nn.conv2d(bottom, filt, [1, 1, 1, 1], padding='SAME')

            conv_biases = self.get_bias(name)
            bias = tf.nn.bias_add(conv, conv_biases)

            relu = tf.nn.relu(bias)
            return relu

    def fc_layer(self, bottom, name):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])

            weights = self.get_fc_weight(name)
            biases = self.get_bias(name)

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            return fc

    def get_conv_filter(self, name):
        return tf.constant(self.data_dict[name][0], name="filter")

    def get_bias(self, name):
        return tf.constant(self.data_dict[name][1], name="biases")

    def get_fc_weight(self, name):
        return tf.constant(self.data_dict[name][0], name="weights")

SyntaxError: invalid syntax (<ipython-input-1-ad1a801d5487>, line 1)